In [31]:
import pandas as pd
basepath=r"C:\Users\krawc\OneDrive\Documents\code\pythons\predictor\data\\"

odds = pd.read_csv(basepath+"match_w_odds_1923.csv", index_col=[0,1,2])


In [32]:
# cols_pin=['PSH', 'PSD', 'PSA']
# cols_wh=['WHH', 'WHD', 'WHA']

# def probs_form_odds(row):    
#     import shin
#     h,d,a=row
#     res=shin.calculate_implied_probabilities([h,d,a])
#     return res['implied_probabilities']

# def apply_probs_form_odds(df,three_cols):
#     df[['pAvgH', 'pAvgD', 'pAvgA']]=df[three_cols].apply(probs_form_odds,axis=1,result_type='expand')

# apply_probs_form_odds(odds,cols_pin)

In [33]:
def add_key_col(hist):
    ew_game=hist['date'] +' ' + hist['team']+'-' + hist['opponent']
    # ew_game=hist['date'].dt.date.astype('string') +' ' + hist['home_team']+'-' + hist['away_team']
    return ew_game

def prep_matches(matches):
    # matches=matches.drop('notes',axis=1)
    matches=matches.iloc[:,:10]
    # matches["date"] = pd.to_datetime(matches["date"])
    matches["target"] = (matches["result"] == "W").astype("int")
    matches["venue_code"] = matches["venue"].astype("category").cat.codes
    # matches["opp_code"] = matches["opponent"].astype("category").cat.codes
    # matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
    # matches["day_code"] = matches["date"].dt.dayofweek
    return matches

def eliminate_outliers(matches,odd_cols,trsh=0.25):
    c1=matches.target==1
    c2=matches.pAvgH<trsh
    matches=matches[~(c1&c2)]#['pAvgH'].quantile(.2)
    #remove odds
    # matches=matches.iloc[:,:-6]
    matches=matches.drop(odd_cols,axis=1)
    return matches

def keysplit(text):
    sd,st=text.split(' ',1)
    sh,sa=st.split('-',1)
    return sd,sh,sa

def reverse_key(oldkey):
    sd,sh,sa = keysplit(oldkey)
    return sd + ' ' + sa + '-' + sh

def reverse_odds_df(odds2):
    odds3=odds2.copy()
    odds3.index=odds3.index.map(reverse_key)
    odds3['AvgH'],odds3['AvgA']=odds3['AvgA'],odds3['AvgH']
    odds3['pAvgH'],odds3['pAvgA']=odds3['pAvgA'],odds3['pAvgH']
    return odds3

def prep_odds(odds):
    cols=['AvgH', 'AvgD', 'AvgA','pAvgH','pAvgD','pAvgA']
    odds2=odds.droplevel([0,1]).loc[:,cols] 

    odds3=reverse_odds_df(odds2)
    
    odds23=pd.concat([odds2,odds3])
    return odds23

def merge_matches_odds(matches,odds):
    matches['key']=add_key_col(matches)
    matches=matches.set_index('key')
    
    matches2=matches.join(odds).reset_index(drop=True)
    odd_cols=list(odds.columns)
    return matches2,odd_cols

def rolling_averages(group, cols, new_cols,window=3):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(window, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group
    # # EXAMPLE
    # grouped_matches = matches.groupby("team")
    # group = grouped_matches.get_group("Manchester City").sort_values("date")
    # rolling_averages(group, cols, new_cols)

def add_rolling(matches):
    cols = ['points']
    new_cols = [f"{c}_rolling" for c in cols]

    wdl={'W':3,'D':1,'L':0}
    matches['points']=matches['result'].map(wdl)

    matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
    matches_rolling = matches_rolling.droplevel('team')
    matches_rolling = matches_rolling.reset_index(drop=True)

    matches_rolling = matches_rolling.drop('points',axis=1)
    return matches_rolling 

def merge_matches_metrics(matches,ranked):
    full=matches.set_index('team').join(ranked).reset_index(drop=False,names='team')
    suf='_opo'
    full=full.set_index('opponent').join(ranked,rsuffix=suf).reset_index(drop=False,names='opponent')
    team_cols=list(ranked.columns)
    opo_cols=[x+suf for x in team_cols]
    return full,team_cols,opo_cols

def calc_diffs(full,team_cols,opo_cols):
    # rankedlen=17
    # startcol=12+0
    
    # sc=startcol
    # rl=rankedlen

    # team_cols=full.columns[sc:sc+rl]
    # opo_cols=full.columns[sc+rl:sc+rl+rl]
    new_names=[x +'_diff' for x in team_cols]
    
    diff_vals=full[team_cols].values-full[opo_cols].values
    diff_df=pd.DataFrame(diff_vals,index=full[team_cols].index,columns=new_names)

    full=full.join(diff_df)
    return full

In [34]:

def run(season,odds):
    matches = pd.read_csv(basepath+f"matches{season}.csv", index_col=0)
    ranked = pd.read_csv(basepath+f"ranked_mets_{season}back.csv", index_col=[0])

    matches=prep_matches(matches)
    odds=prep_odds(odds)

    matches2,odd_cols=merge_matches_odds(matches,odds)

    full,team_cols,opo_cols=merge_matches_metrics(matches2,ranked)
    full=calc_diffs(full,team_cols,opo_cols)
    # full=add_rolling(full)

    full_w_outs=full.copy()
    full=eliminate_outliers(full,odd_cols)
    return full,full_w_outs

seasons=[1920,2021,2122,2223]
lst1=[]
lst2=[]
for season in seasons:
    full,full_w_outs=run(season,odds)
    lst1.append(full)
    lst2.append(full_w_outs)

In [35]:
full=pd.concat(lst1)
full_w_outs=pd.concat(lst2)


In [36]:
def apply_add_key(df):
    df['key']=add_key_col(df)
    df=df.set_index('key')
    return df

full_w_outs=apply_add_key(full_w_outs)
full=apply_add_key(full)

In [37]:
full_w_outs.to_csv("full_w_outs.csv")
full.to_csv("full.csv")